**When the data is huge; it takes so much time to retrieve all the records for a user. Libraries such as datatable makes it faster or if you know all the index values (which is difficult).**

**But I will show another approach so that we can update a user's record.**

In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)


In [ ]:
%%time

dtypes = {
    "row_id":"int64",
    "timestamp":"int64",
    "user_id":"int32",
    "content_id":"int16",
    "content_type_id":"int16",
    "task_container_id":"int16",
    "user_answer":"int8",
    "answered_correctly":"int8",
    "prior_question_elapsed_time":"float32",
    "prior_question_had_explanation":"str"
}

data = pd.read_csv("/kaggle/input/riiid-test-answer-prediction/train.csv", dtype=dtypes)

Let's check in how much time it consumes to retrieve a user's df with .loc

In [ ]:
sample_user = int(data.sample(1, random_state=42)["user_id"])

%time data.loc[data.user_id == sample_user]

But, using .iloc absolutely so much faster. In order to establish my implement I will firstly use .iloc

In [ ]:
%time user_indexes = data.groupby("user_id")["row_id"].agg(["min", "max"]).reset_index()

In [ ]:
from tqdm import tqdm

total_dict = {}
for user in tqdm(range(len(user_indexes))):
    user_df = data.iloc[user_indexes.iloc[user, 1]:user_indexes.iloc[user, 2], :]
    total_dict[f'data_{user_indexes.iloc[user, 0]}'] = user_df

Now, we see that using iloc is really fast. But it might get confusing to update index values. Instead, as we see the code above, I created a dictionary that contains of every user's dataframe with keys of user id. Now, retrieving and updating should be easier.

In [ ]:
%timeit data.iloc[5834946:5835038,:]

In [ ]:
%timeit total_dict[f'data_{sample_user}']

So then, retrieving with dictionary keys may be even faster than .iloc and it is easy to update (you may want to append new rows etc.)